<a href="https://colab.research.google.com/github/bleamer/eva5/blob/master/w4/MNIST-Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.batch1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(0.1)        

        ## Increase the no. of kernels / transition layer
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batch2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(0.1)        

        self.pool1 = nn.MaxPool2d(2, 2)
        
        ## Increase the no. of kernels / transition layer
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batch3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout(0.1)        

        ## Increase the no. of kernels / transition layer
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batch4 = nn.BatchNorm2d(256)
        self.dropout4 = nn.Dropout(0.1)        

        self.pool2 = nn.MaxPool2d(2, 2)
        
        ## Increase the no. of kernels / transition layer
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.batch5 = nn.BatchNorm2d(512)
        self.dropout5 = nn.Dropout(0.1)        
        

        ## Increase the no. of kernels / transition layer
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.batch6 = nn.BatchNorm2d(1024)
        self.dropout6 = nn.Dropout(0.1)        

        ## Reduce the no. of kernels / to required classes
        # self.gap1 = nn.AvgPool2d(kernel_size=(2,2))
        self.conv7 = nn.Conv2d(1024, 10, 3)



    def forward(self, x):
        x = F.relu(self.dropout1(self.batch1(self.conv1(x))))
        x = F.relu(self.dropout2(self.batch2(self.conv2(x))))
        x = self.pool1(x)

        x = F.relu(self.dropout3(self.batch3(self.conv3(x))))
        x = F.relu(self.dropout4(self.batch4(self.conv4(x))))
        x = self.pool2(x)
        # x = self.gap1(x)

        x = F.relu(self.dropout5(self.batch5(self.conv5(x))))
        x = F.relu(self.dropout6(self.batch6(self.conv6(x))))

        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]             512
          Dropout-13          [-1, 256, 14, 14]               0
        MaxPool2d-14            [-1, 25

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 8):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0310, Accuracy: 9904/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0441, Accuracy: 9860/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0247, Accuracy: 9932/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0367, Accuracy: 9892/10000 (99%)



loss=0.0022123297676444054 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.09it/s]



Test set: Average loss: 0.0183, Accuracy: 9940/10000 (99%)

